In [121]:
from dotenv import load_dotenv
import os
from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated, Sequence,List,Union
from langchain_core.messages import BaseMessage, SystemMessage, HumanMessage, ToolMessage,AIMessage,AnyMessage
from operator import add as add_messages
from langchain_openai import ChatOpenAI
from langchain_groq import ChatGroq
from langchain_openai import OpenAIEmbeddings
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma
from langchain_core.tools import tool
from langchain_community.tools import DuckDuckGoSearchRun


In [7]:
load_dotenv()

True

 Smart Research and Summarization Agent with Feedback Loop:

Concept: This agent would go beyond a simple RAG (Retrieval Augmented Generation) system. It would perform web research on a given topic, summarize the findings, and then have a "reflection" or "critique" agent review the summary. If the critique agent finds the summary lacking (e.g., missing key details, not answering the user's specific sub-questions), it would trigger further research and refinement.

In [33]:
GROQ_API_KEY = os.getenv('GROQ_API_KEY')
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
llm = ChatOpenAI(model="gpt-4o-mini")


In [133]:
#Define the LangGraph State
class GraphState(TypedDict):
    """
    Represents the state of our graph.

    Attributes:
        search: input search query by the user.
        summary: summary generated by the research agent.
        review: review performed by the review agent.
        error: Any error message encountered during processing.
    """
    input_str: str
    #web_search: Annotated[str, "results from web search"]
    summary: str
    #messages: Annotated[list[AnyMessage], add_messages]
    messages : List[Union[HumanMessage,AIMessage]]
    #review: Annotated[str, "The structured summarized content"] # Updated type
    #error: Annotated[str, "Any error message encountered"]

In [134]:
#define nodes

def search_summary_text(state:GraphState)->GraphState:
    """
    Takes the input from user and performs web search

    Args:
        state: The state of the Graph containg search string
    Returns:
        Update state of the graph with search results.
    """
    print("state",state)
    llm=ChatGroq(model="llama3-70b-8192")
    search = DuckDuckGoSearchRun()
    tools=[search]
    llm_with_tool = llm.bind_tools(tools)
    input_str = state["input_str"]
    # Prompt Template
    prompt_template = f"""Search web for the query and generate concise and accurate summary in less than 50 words.
    Query: {input_str}
    """
    response = llm_with_tool.invoke([SystemMessage(content=prompt_template)] +
                          [HumanMessage(content="Provide the response.")])
    # Return a new state with the summary added
    #state = dict(state)
    state['input_str']= input_str
    state["summary"] = response.content
    state["messages"] = response
    return state
    
def analyse_summary(state:GraphState)->GraphState:
    """
    Analyse the summary generated by the agent and provide review

    Args:
        state: The state of the Graph containg search string
    Returns:
        Update state of the graph with search results.
    """
    print("analyzing summary",state)
    llm=ChatGroq(model="llama3-70b-8192")
    input_summary = state["summary"]
    input_str = state["input_str"]
    # Prompt Template
    prompt_template = f"""Analyse the input string and the summary and confirm if the summary is accurate and relevant.
    If the summary is accurate, return "Summary is accurate". If not, provide a detailed review of the summary.
    Input String: {input_str}
    Summary: {input_summary}
    """
    response = llm.invoke([SystemMessage(content=prompt_template)] +
                          [HumanMessage(content="Provide the response.")])
    # Return a new state with the review added
    print("response",response.content)
    #new_state = dict(state)
    state["review"] = response.content
    #state['input_str']= input_str
    #state["summary"] = input_summary
    #print("new state",state)
    state["messages"] = [response]
    return state
    


In [135]:
work = StateGraph(GraphState)
work.add_node("search_summary", search_summary_text)
work.add_node("analyse_summary", analyse_summary)
work.set_entry_point("search_summary")
work.add_edge("search_summary","analyse_summary")
work.add_edge("analyse_summary", END)
app = work.compile()

In [136]:
initial_state = {
    "input_str": "What is langgraph?",
    "summary": "",
    "review": "",  # Initialize review if needed,
    "messages": []  # Initialize messages as an empty list
    
}
response = app.invoke(initial_state)

state {'input_str': 'What is langgraph?', 'summary': '', 'messages': []}
analyzing summary {'input_str': 'What is langgraph?', 'summary': 'Langgraph is a graph-based representation of programming languages, allowing for efficient analysis and manipulation of source code.', 'messages': AIMessage(content='Langgraph is a graph-based representation of programming languages, allowing for efficient analysis and manipulation of source code.', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 941, 'total_tokens': 963, 'completion_time': 0.135937402, 'prompt_time': 0.031529108, 'queue_time': 0.206325984, 'total_time': 0.16746651}, 'model_name': 'llama3-70b-8192', 'system_fingerprint': 'fp_dd4ae1c591', 'finish_reason': 'stop', 'logprobs': None}, id='run--30e93d49-0091-4118-b008-1a4065da54f2-0', usage_metadata={'input_tokens': 941, 'output_tokens': 22, 'total_tokens': 963})}
response Summary is accurate.


In [137]:
response


{'input_str': 'What is langgraph?',
 'summary': 'Langgraph is a graph-based representation of programming languages, allowing for efficient analysis and manipulation of source code.',
 'messages': [AIMessage(content='Summary is accurate.', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 5, 'prompt_tokens': 95, 'total_tokens': 100, 'completion_time': 0.048424157, 'prompt_time': 0.002719214, 'queue_time': 0.218770152, 'total_time': 0.051143371}, 'model_name': 'llama3-70b-8192', 'system_fingerprint': 'fp_dd4ae1c591', 'finish_reason': 'stop', 'logprobs': None}, id='run--4297a950-620a-4380-af64-b8337ceb67f3-0', usage_metadata={'input_tokens': 95, 'output_tokens': 5, 'total_tokens': 100})]}